# Level 3: Advanced Agentic Capabilities with Prompt Chaining and ReAct Agent

Building on the simple agent introduced in Level 2, this tutorial continues the agent-focused section of the series by introducing techniques that make the agent smarter and more autonomous: **Prompt Chaining** and the **ReAct (Reasoning + Acting) framework**. These approaches allow the agent to reason through multi-step problems, dynamically choose tools, and adjust its behavior based on context.

- **Prompt Chaining** connects multiple prompts into a coherent sequence, allowing the agent to maintain context and perform multi-step reasoning across tool invocations. [Learn more →](https://www.ibm.com/think/topics/prompt-chaining)
- **ReAct Agent** combines reasoning and acting steps in a loop, enabling the agent to make decisions, use tools dynamically, and adapt based on intermediate results. [Learn more →](https://www.ibm.com/think/topics/react-agent)

## Overview

In this notebook, you'll explore three agent configurations:
1. **Simple Agent (Baseline)** – Uses a single web search tool.
2. **Prompt Chaining** – Performs structured, multi-step reasoning by chaining prompts and responses.
3. **ReAct Agent** – Dynamically plans and executes actions using a loop of reasoning and tool use.


## Prerequisites

Before starting, ensure you have the following:
- Access to a remote cluster or a local Podman setup.
- A Tavily API key is required. You can register for one at [https://tavily.com/](https://tavily.com/).


## Setting the Environment Variables

Use the [`.env.example`](../../../.env.example) to create a new file called `.env` and ensure you add all the relevant environment variables below.

In addition to the environment variables listed in the ["Getting Started" notebook](demos/rag_agentic/notebooks/Level0_getting_started_with_Llama_Stack.ipynb), the following should be provided for this demo to run:
 - `TAVILY_SEARCH_API_KEY`: your API key for tavily search.

## 1. Setting Up the Environment
The geocoder package is required for this demo.

In [1]:
# !pip install geocoder

We will start with a few imports needed for this demo only.

In [2]:
from llama_stack_client.lib.agents.agent import Agent
from llama_stack_client.lib.agents.event_logger import EventLogger
from llama_stack_client.lib.agents.react.agent import ReActAgent
from llama_stack_client.lib.agents.react.tool_parser import ReActOutput
import sys
sys.path.append('..') 
from src.client_tools import get_location

Next, we will initialize our environment as described in detail in our ["Getting Started" notebook](demos/rag_agentic/notebooks/Level0_getting_started_with_Llama_Stack.ipynb). Please refer to it for additional explanations.

In [3]:
# for accessing the environment variables
import os
from dotenv import load_dotenv
load_dotenv()

# for communication with Llama Stack
from llama_stack_client import LlamaStackClient

# pretty print of the results returned from the model/agent
import sys
sys.path.append('..')  
from src.utils import step_printer
from termcolor import cprint

remote = os.getenv("REMOTE", "True")

if remote == "False":
    local_port = os.getenv("LOCAL_SERVER_PORT", 8321)
    base_url = f"http://localhost:{local_port}"
else: # any value non equal to 'False' will be considered as 'True'
    base_url = os.getenv("REMOTE_BASE_URL")


# Tavily search API key is required for some of our demos and must be provided to the client upon initialization.
# We will cover it in the agentic demos that use the respective tool. Please ignore this parameter for all other demos.
tavily_search_api_key = os.getenv("TAVILY_SEARCH_API_KEY")
if tavily_search_api_key is None:
    provider_data = None
else:
    provider_data = {"tavily_search_api_key": tavily_search_api_key}


client = LlamaStackClient(
    base_url=base_url,
    provider_data=provider_data
)
    
print(f"Connected to Llama Stack server")

# model_id will later be used to pass the name of the desired inference model to Llama Stack Agents/Inference APIs
model_id = os.getenv("INFERENCE_MODEL_ID")

temperature = float(os.getenv("TEMPERATURE", 0.0))
if temperature > 0.0:
    top_p = float(os.getenv("TOP_P", 0.95))
    strategy = {"type": "top_p", "temperature": temperature, "top_p": top_p}
else:
    strategy = {"type": "greedy"}

max_tokens = int(os.getenv("MAX_TOKENS", 4096))

# sampling_params will later be used to pass the parameters to Llama Stack Agents/Inference APIs
sampling_params = {
    "strategy": strategy,
    "max_tokens": max_tokens,
}

stream_env = os.getenv("STREAM", "False")
# the Boolean 'stream' parameter will later be passed to Llama Stack Agents/Inference APIs
# any value non equal to 'False' will be considered as 'True'
stream = (stream_env != "False")

print(f"Inference Parameters:\n\tModel: {model_id}\n\tSampling Parameters: {sampling_params}\n\tstream: {stream}")

Connected to Llama Stack server
Inference Parameters:
	Model: granite32-8b
	Sampling Parameters: {'strategy': {'type': 'greedy'}, 'max_tokens': 4096}
	stream: False


## 2. Simple Agent (Baseline)
Same agent setup as Level 2 notebook. 

In [4]:
agent = Agent(
    client, 
    model=model_id,
    instructions="""You are a helpful websearch assistant. When you are asked to search the web you must use a tool. 
            Whenever a tool is called, be sure return the response in a friendly and helpful tone.
            """ ,
    tools=["builtin::websearch"],
    sampling_params=sampling_params
)
user_prompts = [
    "Are there any weather-related risks in my area that could disrupt network connectivity or system availability?",
]
for prompt in user_prompts:
    print("\n"+"="*50)
    cprint(f"Processing user query: {prompt}", "blue")
    print("="*50)
    session_id = agent.create_session("web-session")
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream
    )
    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 


Processing user query: Are there any weather-related risks in my area that could disrupt network connectivity or system availability?

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: brave_search, Arguments: {'query': 'weather-related risks in current location'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


{
│   'query': 'weather-related risks in current location',
│   'top_k': [
│   │   {
│   │   │   'title': 'Weather in current location',
│   │   │   'url': 'https://www.weatherapi.com/',
│   │   │   'content': "{'location': {'name': 'Current', 'region': 'Harbour Island', 'country': 'Bahamas', 'lat': 25.4333, 'lon': -76.7833, 'tz_id': 'America/Nassau', 'localtime_epoch': 1746201687, 'localtime': '2025-05-02 12:01'}, 'current': {'last_updated_epoch': 1746201600, 'last_updated': '2025-05-02 12:00', 'temp_c': 23.5, 'temp_f': 74.3, 'is_day': 1, 'condition': {'text': 'Partly Cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 14.1, 'wind_kph': 22.7, 'wind_degree': 95, 'wind_dir': 'E', 'pressure_mb': 1018.0, 'pressure_in': 30.07, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 64, 'cloud': 26, 'feelslike_c': 25.3, 'feelslike_f': 77.6, 'windchill_c': 23.5, 'windchill_f': 74.3, 'heatindex_c': 25.3, 'heatindex_f': 77.6, 'dewpoint_c': 16.7, 'dewpoint_f': 62.1, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 9.9, 'gust_mph': 19.3, 'gust_kph': 31.1}}",
│   │   │   'score': 0.9286297,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Storm Prediction Center Feb 5, 2025 1300 UTC Day 1 Convective Outlook',
│   │   │   'url': 'https://www.spc.noaa.gov/products/outlook/archive/2025/day1otlk_20250205_1300.html',
│   │   │   'content': 'Severe weather, tornado, thunderstorm, fire weather, storm report, tornado watch, severe thunderstorm watch, mesoscale discussion, convective outlook products from the Storm Prediction Center. ... Day 1 Hail Risk: Area (sq. mi.) Area Pop. Some Larger Population Centers in Risk Area: 5 %: ... Mosier/Bentley.. 02/05/2025 CLICK TO GET',
│   │   │   'score': 0.6388584,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Severe Weather Advisory - February 5, 2025 » Corporate AccuWeather',
│   │   │   'url': 'https://corporate.accuweather.com/newsroom/severe-weather-advisories/severe-weather-advisory-february-5-2025/',
│   │   │   'content': 'Press Release. Wildfires are expected to scorch more land across America this year compared to the historical average. AccuWeather Global Weather Center - April 23, 2025 - In the wake of unprecedented and devastating wildfires that caused an estimated $250 billion to $275 billion in total damage and economic loss across Southern California in January, AccuWeather long-range experts are',
│   │   │   'score': 0.6244086,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Spring Outlook: Dry in the West, milder than average in the South and ...',
│   │   │   'url': 'https://www.noaa.gov/news-release/spring-outlook-dry-in-west-milder-than-average-in-south-and-east',
│   │   │   'content': "Today, forecasters from NOAA's Climate Prediction Center — part of the National Weather Service — issued the 2025 U.S. Spring Outlook for temperature, precipitation, drought and flooding for the coming months.. Forecasters predict above-average temperatures for the East and Gulf Coast regions, while drought conditions continue in the Southwest.",
│   │   │   'score': 0.33222425,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Significant Impact Expected from A Powerful Winter Storm; Mid-Atlantic ...',
│   │   │   'url': 'https://www.nationalweatherforce.com/2025/02/05/significant-impact-expected-from-a-powerful-winter-storm-mid-atlantic-region-feb-5th-2025/',
│   │   │   'content': 'Overall Forecast: The Mid-Atlantic region is bracing for a complex winter weather system that is expected to bring freezing rain and icy conditions starting Wednesday night and continuing into Thursday.This event is likely to affect a wide area, from northern Virginia through to southern New Jersey, impacting major urban centers along the I-95 corridor.',
│   │   │   'score': 0.15330847,
│   │   │   'raw_content': None
│   │   }
│   ]
}


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
Based on the latest weather updates, there are no immediate severe weather warnings or risks in your area that could disrupt network connectivity or system availability. 

1. The current weather in your location (Harbour Island, Bahamas) is partly cloudy with a temperature of 74.3°F (23.5°C). There are no significant weather events predicted in the near future.

2. The Storm Prediction Center's Day 1 Convective Outlook for February 5, 2025, doesn't indicate any severe weather threats in your area.

3. AccuWeather's press release about potential wildfires doesn't specify any immediate risks for your location.

4. NOAA's Spring Outlook predicts above-average temperatures for the East and Gulf Coast regions, but this is a broader forecast and doesn't indicate immediate risks for your area.

5. A winter storm is expected to affect the Mid-Atlantic region, but this doesn't seem to impact your location.

Please note that weathe

### Output Analysis

In this example, the agent hallucinates and incorrectly identifies the user's location—despite the notebook running from Dublin, Ireland. This misidentification leads to inaccurate information about potential weather-related risks.

This is where Prompt Chaining comes in. Prompt chaining allows the agent to:
1. Maintain context across multiple queries
2. Chain multiple tools together
3. Use previous interactions to inform current decisions

Let’s see how prompt chaining can improve the accuracy of the response.

## 3. Prompt chaining with websearch tool and client tool

In this section, we demonstrate a more sophisticated use case that combines location detection with web search. Instead of explicitly providing location information, we'll:

1. **Automatic Location Detection**: Use the `get_location` client tool to automatically determine the user's current location
2. **Contextual Search**: Leverage the detected location to find relevant nearby places

For example, when a user asks "Are there any weather-related risks in my area that could disrupt network connectivity or system availability?", the agent will:
- First detect the user's current location
- Then use that location to search for nearby weather-related risks
- Present a comprehensive response

This demonstrates how builtin websearch tool and client tool can work together to provide intelligent, context-aware responses without requiring explicit location input from the user.

In [5]:
agent = Agent(
    client, 
    model=model_id,
    instructions="""You are a helpful assistant. 
    When a user asks about their location, you MUST use the get_location tool. When searching for nearby places, you MUST use the websearch tool.
    """ ,
    tools=[get_location, "builtin::websearch"],
    sampling_params=sampling_params
)
user_prompts = [
    "Where am I?",
    "Are there any weather-related risks in my area that could disrupt network connectivity or system availability?"
]
session_id = agent.create_session("prompt-chaining-session")  # for prompt chaining, queries must share the same session_id.
for prompt in user_prompts:
    print("\n"+"="*50)
    cprint(f"Processing user query: {prompt}", "blue")
    print("="*50)
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream
    )

    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 


Processing user query: Where am I?

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: get_location, Arguments: {'query': 'current location'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


'Your current location is: Dublin, Leinster, IE'


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
You are currently in Dublin, Leinster, Ireland.

========== Query processing completed ========== 


Processing user query: Are there any weather-related risks in my area that could disrupt network connectivity or system availability?

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: brave_search, Arguments: {'query': 'weather risks in Dublin, Leinster, Ireland'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


{
│   'query': 'weather risks in Dublin, Leinster, Ireland',
│   'top_k': [
│   │   {
│   │   │   'title': 'Weather in Dublin, Leinster, Ireland',
│   │   │   'url': 'https://www.weatherapi.com/',
│   │   │   'content': "{'location': {'name': 'Dublin', 'region': 'Dublin', 'country': 'Ireland', 'lat': 53.3331, 'lon': -6.2489, 'tz_id': 'Europe/Dublin', 'localtime_epoch': 1746201697, 'localtime': '2025-05-02 17:01'}, 'current': {'last_updated_epoch': 1746201600, 'last_updated': '2025-05-02 17:00', 'temp_c': 15.0, 'temp_f': 59.0, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 8.1, 'wind_kph': 13.0, 'wind_degree': 8, 'wind_dir': 'N', 'pressure_mb': 1019.0, 'pressure_in': 30.09, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 59, 'cloud': 50, 'feelslike_c': 14.3, 'feelslike_f': 57.7, 'windchill_c': 12.5, 'windchill_f': 54.5, 'heatindex_c': 13.6, 'heatindex_f': 56.4, 'dewpoint_c': 7.1, 'dewpoint_f': 44.7, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 2.0, 'gust_mph': 9.5, 'gust_kph': 15.2}}",
│   │   │   'score': 0.8745478,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': "Weather warnings 'likely' as heavy rain to batter Ireland",
│   │   │   'url': 'https://www.dublinlive.ie/news/met-eireann-ireland-weather-change-31424095',
│   │   │   'content': 'Met Eireann is forecasting the risk of hail and thunderstorms during the day followed by persistent rain moving across much of Munster and Leinster that night. Weather Alerts Ireland said: "A low',
│   │   │   'score': 0.76002824,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Dublin weather: Met Eireann predicts big change on horizon within hours ...',
│   │   │   'url': 'https://www.dublinlive.ie/news/dublin-news/dublinweather-forecast-ireland-today-tomorrow-31546785',
│   │   │   'content': 'Some well scattered showers will develop, mostly likely over Leinster and parts of Munster. Highest temperatures of 11 to 15 degrees generally but 9 to 11 degrees in Ulster, with light to moderate',
│   │   │   'score': 0.58976424,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Ireland weather: Scorching temps to grip country in ... - Dublin Live',
│   │   │   'url': 'https://www.dublinlive.ie/news/weather-ireland-summer-met-eireann-31556228',
│   │   │   'content': "Ireland is set to sizzle in scorching temperatures this May. The country will see temperatures soar to above average, according to Met Eireann's extended range forecast.",
│   │   │   'score': 0.36833808,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Dublin, Leinster 30-day outlook | 30-day weather forecast | monthly ...',
│   │   │   'url': 'https://www.myforecast.com/30-day-forecast.php?cwid=66185&condition=act',
│   │   │   'content': "MyForecast is a comprehensive resource for online weather forecasts and reports for over 72,000 locations worldcwide. You'll find detailed 48-hour and 7-day extended forecasts, ski reports, marine forecasts and surf alerts, airport delay forecasts, fire danger outlooks, Doppler and satellite images, and thousands of maps. ... Dublin, Leinster",
│   │   │   'score': 0.35374123,
│   │   │   'raw_content': None
│   │   }
│   ]
}


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
According to the latest weather forecasts, there are no immediate weather-related risks in Dublin, Leinster, Ireland that could disrupt network connectivity or system availability. However, there is a forecast of heavy rain and potential thunderstorms, which could impact outdoor activities or travel plans. It's always a good idea to stay updated with the local weather forecast for any changes.

========== Query processing completed ========== 



### ReAct Agent with websearch tool and client tool

This section demonstrates the ReAct (Reasoning and Acting) framework in action.

For example, when asked "Are there any weather-related risks in my area that could disrupt network connectivity or system availability?", the agent:
1. **Reasons** it needs location information first
2. **Acts** by calling the `get_location` client tool
3. **Observes** the location result
4. **Reasons** about the next step
5. **Acts** by calling the `websearch` tool with observed location
6. **Observes** and processes the search results

Unlike prompt chaining which follows fixed steps, ReAct dynamically breaks down tasks and adapts its approach based on the results of each step. This makes it more flexible and capable of handling complex, real-world queries effectively.

In [6]:
agent = ReActAgent(
            client=client,
            model=model_id,
            tools=[get_location, "builtin::websearch"],
            response_format={
                "type": "json_schema",
                "json_schema": ReActOutput.model_json_schema(),
            },
            sampling_params=sampling_params,
        )
user_prompts = [
    "Are there any weather-related risks in my area that could disrupt network connectivity or system availability?"
]
session_id = agent.create_session("web-session")
for prompt in user_prompts:
    print("\n"+"="*50)
    cprint(f"Processing user query: {prompt}", "blue")
    print("="*50)
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream
    )
    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 


Processing user query: Are there any weather-related risks in my area that could disrupt network connectivity or system availability?

---------- 📍 Step 1: InferenceStep ----------
🤖 Model Response:
{
    "thought": "I need to find out the user's current location and then check for any weather-related risks that could disrupt network connectivity or system availability. I will use the `get_location` tool to get the user's location and then the `web_search` tool to find any weather-related risks.",
    "action": {
        "tool_name": "get_location",
        "tool_params": [{"name": "query", "value": "user's current location"}]
    },
    "answer": null
}


---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


'Your current location is: Dublin, Leinster, IE'


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
{
    "thought": "Now that I have the user's location, I will use the `web_search` tool to find any weather-related risks that could disrupt network connectivity or system availability in Dublin, Leinster, IE.",
    "action": {
        "tool_name": "web_search",
        "tool_params": [{"name": "query", "value": "weather-related risks disrupting network connectivity or system availability in Dublin, Leinster, IE"}]
    },
    "answer": null
}


---------- 📍 Step 4: ToolExecutionStep ----------
🔧 Executing tool...


{
│   'query': 'weather-related risks disrupting network connectivity or system availability in Dublin, Leinster, IE',
│   'top_k': [
│   │   {
│   │   │   'title': 'Weather in Dublin, Leinster, IE',
│   │   │   'url': 'https://www.weatherapi.com/',
│   │   │   'content': "{'location': {'name': 'Dublin', 'region': 'Dublin', 'country': 'Ireland', 'lat': 53.3331, 'lon': -6.2489, 'tz_id': 'Europe/Dublin', 'localtime_epoch': 1746201708, 'localtime': '2025-05-02 17:01'}, 'current': {'last_updated_epoch': 1746201600, 'last_updated': '2025-05-02 17:00', 'temp_c': 15.2, 'temp_f': 59.4, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 8.1, 'wind_kph': 13.0, 'wind_degree': 8, 'wind_dir': 'N', 'pressure_mb': 1019.0, 'pressure_in': 30.09, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 59, 'cloud': 50, 'feelslike_c': 15.2, 'feelslike_f': 59.4, 'windchill_c': 12.5, 'windchill_f': 54.5, 'heatindex_c': 13.6, 'heatindex_f': 56.4, 'dewpoint_c': 7.1, 'dewpoint_f': 44.7, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 2.0, 'gust_mph': 9.5, 'gust_kph': 15.2}}",
│   │   │   'score': 0.4499919,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': "Ireland's infrastructure is vulnerable to extreme weather",
│   │   │   'url': 'https://www.tcd.ie/e3/news/irelands-infrastructure-is-vulnerable-to-extreme-weather/',
│   │   │   'content': "Ireland's infrastructure is vulnerable to extreme weather 03 Feb 2025 This Op Ed by Dr. Muhammad Ali, Prof. Brian Caulfield, and Dr. Julie Clarke from the the department of civil, structural and environmental engineering at Trinity College Dublin, highlights Ireland's vulnerability to extreme weather events, as evidenced by Storm Éowyn.",
│   │   │   'score': 0.40910333,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': "'Lessons to be learned' from Éowyn ahead of future storms - RTÉ",
│   │   │   'url': 'https://www.rte.ie/news/analysis-and-comment/2025/0221/1497988-storm-eowyn-weather/',
│   │   │   'content': 'Storm Éowyn was a real eye opener about the damage an extreme weather event can cause in Ireland and highlighted key vulnerabilities in electricity, water, and telecommunications, writes George Lee.',
│   │   │   'score': 0.302474,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Impact Analysis: Storm Éowyn Tests the Resilience of Telecoms ... - Ookla',
│   │   │   'url': 'https://www.ookla.com/articles/storm-eowyn',
│   │   │   'content': 'In Scotland and Northern Ireland, mobile download speeds at the 10th percentile—a critical metric reflecting the poorest network performance—dropped significantly on the same day, falling by 63% to 2.19 Mbps and by 74% to 1.31 Mbps, respectively, compared to the 7-day average. In December, Ookla published detailed research on the impact of Storm Darragh, highlighting the unique challenges posed by widespread, prolonged power outages on mobile networks. Analysis of Speedtest Intelligence® data highlights the significant impact of Storm Éowyn on mobile network performance in Ireland. It is notable that the limited penetration of battery backup solutions in Ireland’s mobile site grid—typically providing only four to eight hours of power where available—stands in sharp contrast to regions like the Nordics and Australia, which have implemented comprehensive policy measures to harden telecoms infrastructure, as highlighted in a recent Ookla article.',
│   │   │   'score': 0.2751874,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': "After Storm Éowyn, can Ireland's electricity ... - The Irish Times",
│   │   │   'url': 'https://www.irishtimes.com/environment/climate-crisis/2025/01/31/after-storm-eowyn-can-irelands-electricity-water-and-phone-networks-cope-with-further-extreme-weather/',
│   │   │   'content': 'Ireland\'s electricity, water and telecom providers may expect the report card on their response 


---------- 📍 Step 5: InferenceStep ----------
🤖 Model Response:
{
    "thought": "Based on the search results, there have been instances of extreme weather events, such as Storm \u00c9owyn, that have caused significant disruptions to Ireland's infrastructure, including telecommunications. While the current weather conditions might not pose an immediate risk, it's important to be aware of the potential for disruptions during extreme weather events.",
    "action": null,
    "answer": "There have been instances of extreme weather events, such as Storm \u00c9owyn, that have caused significant disruptions to Ireland's infrastructure, including telecommunications. While the current weather conditions might not pose an immediate risk, it's important to be aware of the potential for disruptions during extreme weather events."
}

========== Query processing completed ========== 



## Key Takeaways
- This tutorial demonstrates how to build smarter agents using Prompt Chaining and the ReAct framework.
- It shows how agents can maintain context across steps and perform structured, multi-step reasoning.
- It highlights how ReAct enables dynamic tool selection and adaptive decision-making based on intermediate results.
- These techniques enhance agent autonomy and make them more suitable for complex operational tasks.

For further extensions, continue exploring the next levels in this notebook series.